In [1]:
import numpy as np
import  pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB

In [2]:
def data_pro_scal():
    NB_halt_words = pd.read_csv(filepath_or_buffer='stoplist.txt', header=None)
    NB_halt_words = NB_halt_words.values.flatten().tolist()
    NB_train_data_features = pd.read_csv(filepath_or_buffer='traindata.txt', header=None)
    NB_train_data_labels = pd.read_csv(filepath_or_buffer='trainlabels.txt', header=None)
    NB_train_da_word_labels = pd.DataFrame(data={'features': NB_train_data_features[0], 'labels': NB_train_data_labels[0]}, columns=['features', 'labels'])
    NB_test_data_features = pd.read_csv(filepath_or_buffer='testdata.txt', header=None)
    NB_testing_data_labels = pd.read_csv(filepath_or_buffer='testlabels.txt', header=None)
    NB_test_word_labels = pd.DataFrame(data={'features': NB_test_data_features[0], 'labels': NB_test_data_features[0]}, columns=['features', 'labels'])
    word_list_dir = NB_train_data_features[0].str.split()
    word_dir_dicn = []
    for i in word_list_dir:
        for j in i:
            if j not in NB_halt_words:
                word_dir_dicn.append(j)   
    word_dir_dicn = sorted(list(set(word_dir_dicn)))

    
    NB_training_max = []
    for i in NB_train_data_features[0].str.split():
        train_feat_vector = []    
        
        for j in word_dir_dicn:
            
            if j in i :
                train_feat_vector.append(1)
            else:
                train_feat_vector.append(0)
        NB_training_max.append(train_feat_vector)      
    train_NB_dict_max = np.vstack(NB_training_max)    
    Train_df_NB = pd.DataFrame(data=train_NB_dict_max, columns=word_dir_dicn)
    Train_df_NB['labels'] = NB_train_data_labels
    
    NB_Max_test = []
    for i in NB_test_data_features[0].str.split():
        NB_fec_vec_test_for = []
        
        for j in word_dir_dicn:
            
            if j in i:
                NB_fec_vec_test_for.append(1)
            else:
                NB_fec_vec_test_for.append(0)
        NB_Max_test.append(NB_fec_vec_test_for)
    Test_new_Mat_NB = np.vstack(NB_Max_test)
    Test_NB_dataframe = pd.DataFrame(data=Test_new_Mat_NB, columns=word_dir_dicn)
    Test_NB_dataframe['labels'] = NB_testing_data_labels
    

    return Train_df_NB, NB_train_data_features[0].tolist(), NB_train_da_word_labels, Test_NB_dataframe

In [3]:
train_data, train_data_feat, train_data_comb, test_data = data_pro_scal()

In [4]:
def co_nb_pro_dataset():

    NB_train_for_msg = train_data_comb[train_data_comb['labels'] == 1]
    NB_non_for_train_data = train_data_comb[train_data_comb['labels'] == 0]
    
    dict_fortune_msg = []
    Dic_NB_non_for_data = []
    
    for i in train_data.columns[:-1]:
        
        count_i_in_fortune = 0
        count_i_in_not_fortune = 0
        
   
        for j in NB_train_for_msg['features']:
            if i in j.split():
                count_i_in_fortune +=1     
        dict_fortune_msg.append( (count_i_in_fortune + 1) / ( NB_train_for_msg.shape[0] + 2) )
        for k in NB_non_for_train_data['features']:
            if i in k.split():
                count_i_in_not_fortune +=1
        Dic_NB_non_for_data.append( (count_i_in_not_fortune +1 )/ ( NB_non_for_train_data.shape[0] + 2) )
        
    return dict_fortune_msg, Dic_NB_non_for_data

In [5]:
def get_prior_prob():
    count_classes = train_data.groupby('labels').size()
    prob_class_0 = count_classes[0] / np.sum(count_classes)
    prob_class_1 = count_classes[1] / np.sum(count_classes)
    
    return prob_class_0, prob_class_1

In [6]:
pc_0, pc_1 = get_prior_prob()

In [7]:
dict_fr_msg, dict_n_fr_msg = co_nb_pro_dataset()

In [8]:
def Count_accu_NB():
    pred_labels = []
    for i in range(len(train_data)):
        
        NB_data_col_1 = train_data.loc[i][:-1] * dict_fr_msg
        NB_data_col_o = train_data.loc[i][:-1] * dict_n_fr_msg 
        
        NB_data_col_1_indx = np.where(NB_data_col_1 != 0)
        NB_data_col_o_indx = np.where(NB_data_col_o != 0)
        
        NB_data_col_1_prob = np.prod(NB_data_col_1[NB_data_col_1_indx[0]]) * pc_1
        NB_data_col_o_prob = np.prod(NB_data_col_o[NB_data_col_o_indx[0]]) * pc_0

        if NB_data_col_1_prob > NB_data_col_o_prob:
            pred_labels.append(1)
        else:
            pred_labels.append(0)
    
    train_data['pred_labels'] = pred_labels
            
    print('Naive Bayes accuracy on training data %.2f %%' % (accuracy_score(train_data['labels'], train_data['pred_labels']) * 100 ))
    
    
    test_labels = []
    for i in range(len(test_data)):
        
        NB_data_col_1 = test_data.loc[i][:-1] * dict_fr_msg
        NB_data_col_o = test_data.loc[i][:-1] * dict_n_fr_msg 
        
        NB_data_col_1_indx = np.where(NB_data_col_1 != 0)
        NB_data_col_o_indx = np.where(NB_data_col_o != 0)
        
        NB_data_col_1_prob = np.prod(NB_data_col_1[NB_data_col_1_indx[0]]) * pc_1
        NB_data_col_o_prob = np.prod(NB_data_col_o[NB_data_col_o_indx[0]]) * pc_0
        
        if NB_data_col_1_prob > NB_data_col_o_prob:
            test_labels.append(1)
        else:
            test_labels.append(0)
    
    test_data['pred_labels'] = test_labels
    print('Naive Bayes accuracy on testing data %.2f %%' % (accuracy_score(test_data['labels'], test_data['pred_labels']) * 100))

In [9]:
Count_accu_NB()

Naive Bayes accuracy on training data 95.34 %
Naive Bayes accuracy on testing data 82.18 %


In [10]:
X_train = train_data.iloc[:,:-2]
Y_train = train_data['labels']
X_test = test_data.iloc[:, :-2]
Y_test = test_data['labels']
nb_model = BernoulliNB().fit(X_train, Y_train)

In [11]:
print('Naive Bayes sklearn accuracy on traning data %.2f %% ' %(accuracy_score(Y_train, nb_model.predict(X_train)) *100 ) )
print('Naive Bayes sklearn accuracy on testing data %.2f %% ' %( accuracy_score(Y_test, nb_model.predict(X_test)) *100 ))

Naive Bayes sklearn accuracy on traning data 93.17 % 
Naive Bayes sklearn accuracy on testing data 77.23 % 
